In [15]:
#main_wiki_htmlとinfo_fir_searchを作成します
import pickle,os
import bs4
from bs4 import BeautifulSoup
import re
pickle_year_lsit = os.listdir("../manga_wiki_html_data/")
title_to_page_name_dict = {}

for pickle_year in pickle_year_lsit:
    with open(f"../manga_wiki_html_data/{pickle_year}","rb") as f:
        load_data = pickle.load(f)
    for title,res in load_data.items():
        # print(res)
        soup = BeautifulSoup(res, 'html.parser')
        page_tilte = soup.find("title").get_text().replace(" - Wikipedia","") #こちらの方がより効率が良さそう
        title_to_page_name_dict[title] = page_tilte 
        print(page_tilte)

with open("../../data_for_search/title_to_page_name_dict.pickle","wb") as f:
    pickle.dump(title_to_page_name_dict,f)